In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Using project root:", PROJECT_ROOT)
print("sys.path[0]:", sys.path[0])


Using project root: d:\Dev\ACM_auto_judge
sys.path[0]: d:\Dev\ACM_auto_judge


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import joblib
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import joblib
df = pd.read_csv("../data/preprocessed_problems.csv")
print(df.shape)
print(df.columns)
df.head()

(4112, 10)
Index(['title', 'description', 'input_description', 'output_description',
       'sample_io', 'problem_class', 'problem_score', 'url', 'sample_io_text',
       'full_text'],
      dtype='object')


,title,description,input_description,output_description,sample_io,problem_class,problem_score,url,sample_io_text,full_text
0,Uuu,Unununium (Uuu) was the name of the chemical\r...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2\n2 3\n1 3\n3...",hard,9.7,https://open.kattis.com/problems/uuu,NaN,Uuu Unununium (Uuu) was the name of the chemic...
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\r\n the positions of t...,"[{'input': '0\n2 3 2\n50 60 50\n30 50 40', 'ou...",hard,9.7,https://open.kattis.com/problems/husbygge,NaN,House Building A number of eccentrics from cen...
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,NaN,NaN,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi,NaN,Mario or Luigi Mario and Luigi are playing a g...
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3\n3 C\n2 C\n1 C', 'output': 'GH...",hard,9.6,https://open.kattis.com/problems/thewireghost,NaN,The Wire Ghost Žofka is bending a copper wire....
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0\n10 0\n10 10', 'output': '14.1...",hard,9.6,https://open.kattis.com/problems/barktree,NaN,Barking Up The Wrong Tree Your dog Spot is let...


In [3]:
import importlib
import utils.feature_engineering as fe
importlib.reload(fe)
print("Reloaded from:", fe.__file__)
print(dir(fe))

Reloaded from: d:\Dev\ACM_auto_judge\utils\feature_engineering.py
['ADVANCED_TOPICS', 'CONSTRAINT_FEATURE_NAMES', 'DIFFICULTY_KEYWORDS', 'STRUCTURE_FEATURE_NAMES', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'algorithmic_depth', 'constraint_features', 'extract_technical_text', 'input_structure_features', 'keyword_features', 'math_symbol_count', 're', 'text_length', 'word_count']


In [4]:
from utils.feature_engineering import (
    text_length,
    word_count,
    math_symbol_count,
    keyword_features,
    DIFFICULTY_KEYWORDS,
    constraint_features,
    CONSTRAINT_FEATURE_NAMES,
    input_structure_features,
    STRUCTURE_FEATURE_NAMES,
    algorithmic_depth
)
import pandas as pd

In [5]:
df["text_length"] = df["full_text"].apply(text_length)
df["word_count"] = df["full_text"].apply(word_count)
df["math_symbol_count"] = df["full_text"].apply(math_symbol_count)
df["algorithmic_depth"] = df["full_text"].apply(algorithmic_depth)

In [6]:
structure_matrix = df["full_text"].apply(input_structure_features).tolist()
structure_df = pd.DataFrame(
    structure_matrix,
    columns=STRUCTURE_FEATURE_NAMES
)

df = pd.concat([df, structure_df], axis=1)


In [7]:
print(df.columns)


Index(['title', 'description', 'input_description', 'output_description',
       'sample_io', 'problem_class', 'problem_score', 'url', 'sample_io_text',
       'full_text', 'text_length', 'word_count', 'math_symbol_count',
       'algorithmic_depth', 'multiple_tests', 'graph_edges', 'matrix_or_grid',
       'tree_structure', 'graph_direction'],
      dtype='object')


In [8]:
print(df[STRUCTURE_FEATURE_NAMES].head())

   multiple_tests  graph_edges  matrix_or_grid  tree_structure  \
0               1            1               0               0   
1               1            0               0               0   
2               0            0               0               0   
3               0            0               0               0   
4               0            0               0               1   

   graph_direction  
0                1  
1                0  
2                0  
3                0  
4                0  


In [9]:
constraint_matrix = df["full_text"].apply(constraint_features).tolist()
constraint_df = pd.DataFrame(
    constraint_matrix,
    columns=CONSTRAINT_FEATURE_NAMES
)

df = pd.concat([df, constraint_df], axis=1)


In [10]:
dense_features = df[
    [
        "text_length",
        "word_count",
        "math_symbol_count",
        "algorithmic_depth",
        "has_1e5",
        "has_1e6",
        "has_2e5",
        "mentions_constraints",
        "mentions_time_limit",
        "multiple_tests",
        "graph_edges",
        "matrix_or_grid",
        "tree_structure",
        "graph_direction"
    ]
].values
print("Dense feature shape:", dense_features.shape)
joblib.dump(dense_features, "../models/X_dense.pkl")

Dense feature shape: (4112, 14)


['../models/X_dense.pkl']

In [11]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=5000,
    min_df=3,
    stop_words="english"
)
X_tfidf = tfidf.fit_transform(df["full_text"])
print("TF-IDF shape:", X_tfidf.shape)

TF-IDF shape: (4112, 5000)


In [12]:
df["text_length"] = df["full_text"].apply(text_length)
df["word_count"] = df["full_text"].apply(word_count)
df["math_symbol_count"] = df["full_text"].apply(math_symbol_count)
keyword_matrix = df["full_text"].apply(keyword_features)
keyword_df = pd.DataFrame(
    keyword_matrix.tolist(),
    columns=[f"kw_{kw.replace(' ', '_')}" for kw in DIFFICULTY_KEYWORDS]
)
constraint_matrix = df["full_text"].apply(constraint_features)
constraint_df = pd.DataFrame(
    constraint_matrix.tolist(),
    columns=CONSTRAINT_FEATURE_NAMES
)
structure_matrix = df["full_text"].apply(input_structure_features)
structure_df = pd.DataFrame(
    structure_matrix.tolist(),
    columns=STRUCTURE_FEATURE_NAMES
)
df["algorithmic_depth"] = df["full_text"].apply(algorithmic_depth)

In [13]:
manual_features = pd.concat(
    [
        df[["text_length", "word_count", "math_symbol_count", "algorithmic_depth"]],
        keyword_df,
        constraint_df,
        structure_df
    ],
    axis=1
)
print("Manual features shape:", manual_features.shape)
manual_features.head()

Manual features shape: (4112, 27)


,text_length,word_count,math_symbol_count,algorithmic_depth,kw_dp,kw_dynamic_programming,kw_graph,kw_tree,kw_dfs,kw_bfs,...,has_1e5,has_1e6,has_2e5,mentions_constraints,mentions_time_limit,multiple_tests,graph_edges,matrix_or_grid,tree_structure,graph_direction
0,1703,285,3,0,0,0,5,0,0,0,...,0,0,0,0,0,1,1,0,0,1
1,1451,239,3,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1360,218,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1415,236,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2300,406,0,0,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
X = joblib.load("../models/X_dense.pkl")
print("Final feature matrix shape:", X.shape)
y_class = df["problem_class"]
y_score = df["problem_score"]
print(y_class.value_counts())
print(y_score.describe())


Final feature matrix shape: (4112, 14)
problem_class
hard      1941
medium    1405
easy       766
Name: count, dtype: int64
count    4112.000000
mean        5.114689
std         2.177770
min         1.100000
25%         3.300000
50%         5.200000
75%         6.900000
max         9.700000
Name: problem_score, dtype: float64


In [15]:
joblib.dump(tfidf, "../models/tfidf_vectorizer_v2.pkl")
joblib.dump(X, "../models/X_features_v2.pkl")
joblib.dump(y_class, "../models/y_class.pkl")
joblib.dump(y_score, "../models/y_score.pkl")
joblib.dump(dense_features, "../models/X_dense.pkl")
print("Artifacts saved successfully")

Artifacts saved successfully
